In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import pickle
import tensorflow_hub as hub
from model_architecture import get_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam


In [2]:
save_path = './'

with open(os.path.join(save_path, "train_data.pickle"), 'rb') as f:
    (train_data, train_labels) = pickle.load(f)

with open(os.path.join(save_path, "test_data.pickle"), 'rb') as f:
    (test_data, test_labels) = pickle.load(f)

In [3]:
print(train_data.shape,train_labels.shape)

(2200, 224, 224, 3) (2200,)


In [4]:
print(test_data.shape,test_labels.shape)

(1000, 224, 224, 3) (1000,)


In [5]:
def normalize_image_numpy(image):
    # Convert the pixel values to floats between 0 and 1
    image = image.astype(np.float32) / 255.0
    return image

train_data = normalize_image_numpy(train_data[:3000])
test_data = normalize_image_numpy(test_data)
train_labels = train_labels[:3000]

In [6]:
validation_split = 0.2  # 20%

# Calculate the number of samples for validation
num_validation_samples = int(validation_split * len(train_data))

# Split the data into training and validation sets
X_validation = train_data[:num_validation_samples]
y_validation = train_labels[:num_validation_samples]

train_data = train_data[num_validation_samples:]
train_labels = train_labels[num_validation_samples:]

In [7]:
train_data.shape

(1760, 224, 224, 3)

### Model

In [8]:
checkpoint_path = "./weights/weights{epoch:04d}.weights.h5"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                   verbose =1,
                                                   save_freq = 150)

custom_optimizer = Adam(learning_rate=0.000001)

In [9]:
# model.save(...) saves even the state of model optimizer
# load with 'keras.models.load_model(...)'

In [10]:
def get_iteration_from_filename(filename):
    return int(filename.split('.')[0][-4:])  # Extracting the numeric part from the filename

def get_max_iteration_and_model(model_path):
    # Get list of all weight files
    weight_files = os.listdir(model_path)

    # Filter out non-weight files and extract iteration numbers
    iteration_weights = [(get_iteration_from_filename(file), file) for file in weight_files if file.endswith('.weights.h5')]

    if iteration_weights:
        # Select the weight file with maximum iteration number
        max_iteration_file = max(iteration_weights, key=lambda x: x[0])[1]
        max_iteration = max(iteration_weights, key=lambda x: x[0])[0]  # Get maximum iteration number
        model = get_model()
        model.summary()
        model.load_weights(os.path.join(model_path, max_iteration_file))
        return max_iteration, model
    else:
        return 0, None  # No weight files found

model_path = './weights/'
max_iteration, model = get_max_iteration_and_model(model_path)

if model is not None:
    print("Loaded model with maximum iteration:", max_iteration)
    model.compile(
        optimizer=custom_optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    input_shape = model.input_shape
    print("Input shape required for the model:", input_shape)
    # Additional code using the loaded model
else:
    model = get_model()
    #model.load_weights('./weights/weights0002.weights.h5')

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │    51,382,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         8,196 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,105,156 (252.17 MB)

 Trainable params: 51,390,468 (196.04 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Loaded model with maximum iteration: 63
Input shape required for the model: (None, 224, 224, 3)


In [11]:
epochs = 1000

In [12]:
model.fit(train_data,
          train_labels,
          epochs=epochs, 
          initial_epoch = max_iteration,
          batch_size= 32,
          validation_data=(X_validation, y_validation),
          verbose = 1,
          callbacks=[cp_callback])

In [13]:
model.evaluate(test_data, test_labels)

32/32 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.8339 - loss: 0.4672


[0.4628533720970154, 0.824999988079071]

In [16]:
y_pred_6 = model.predict(test_data)

32/32 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step


In [17]:
y_pred_6 = np.argmax(y_pred_6, axis=1)

In [18]:
from sklearn.metrics import classification_report
classification_rep_6 = classification_report(test_labels, y_pred_6)
print("Classification Report:")
print(classification_rep_6)

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.84      0.80       193
           1       0.92      0.80      0.86       357
           2       0.80      0.77      0.79       234
           3       0.79      0.90      0.84       216

    accuracy                           0.82      1000
   macro avg       0.82      0.83      0.82      1000
weighted avg       0.83      0.82      0.83      1000

